In [81]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import tensorflow as tf

In [6]:
def reverse_dict_with_duplicates(input_dict):
    reversed_dict = {}
    
    for key, value in input_dict.items():
        if value not in reversed_dict:
            reversed_dict[value] = [key]
        else:
            reversed_dict[value].append(key)
    
    return reversed_dict

In [1]:
letter_dict = {
    "ا": "ا",
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ب": "ب",
    "ت": "ب",
    "ث": "ب",
    "ج": "ح",
    "ح": "ح",
    "خ": "ح",
    "د": "د",
    "ذ": "د",
    "ر": "ر",
    "ز": "ر",
    "س": "س",
    "ش": "س",
    "ص": "ص",
    "ض": "ص",
    "ط": "ط",
    "ظ": "ط",
    "ع": "ع",
    "غ": "ع",
    "ف": "ف",
    "ق": "ف",
    "ك": "ك",
    "ل": "ل",
    "م": "م",
    "ن": "ن",
    "و": "و",
    "ؤ": "و",
    "ه": "ه",
    "ة": "ه",
    "ي": "ى",
    "ى": "ى",
    "ئ": "ى",
    " ": " ",
}
letter_set = set(letter_dict.keys())
iLetters = reverse_dict_with_duplicates(letter_dict)

NameError: name 'reverse_dict_with_duplicates' is not defined

In [8]:
# Replaces multiple spaces, or whatever char designated by the perameter into one char. (helper function)
def replace(string, char):
    while char+char in string:
        string = string.replace(char+char, char)

    return string

In [9]:
def parse_text(text):
    
  text = text.strip(" ")
  
  clean_chars = [char for char in text if char in letter_set]
  clean_string = ''.join(clean_chars)

  dotless_chars = [letter_dict[char] for char in clean_chars]
  dotless_string = ''.join(dotless_chars)
      
  clean_string = replace(clean_string, ' ')
  dotless_string = replace(dotless_string, '_')
  
  return {"clean": clean_string, "dotless": dotless_string}

In [10]:
#Arabic wordlist into set
#with open('arabic-wordlist-1.6.txt', 'r') as f:
   # dictionary = set(f.read().splitlines())


In [11]:
def allVariation(root, check, i = 0 ):
    tWord = []
    word = [a for a in root]
    char = word[i]
    for dot in iLetters[char]:
        word[i] = dot
        vWord = ''.join(word)
        if i != len(word) - 1:
            tWord.extend(allVariation(vWord, check, i+1))
        else:
            if check == False:
                tWord.append(vWord)
            elif vWord in dictionary:
                tWord.append(vWord)
    return tWord

In [12]:
def find_different_indices(str1, str2):
    # Initialize an empty list to store the different indices
    different_indices = []

    # Iterate through the characters of the strings
    for i in range(min(len(str1), len(str2))):
        if str1[i] != str2[i]:
            different_indices.append(i)

    # If one string is longer than the other, consider the remaining characters as different
    for i in range(min(len(str1), len(str2)), max(len(str1), len(str2))):
        different_indices.append(i)

    return different_indices

In [32]:
pipe = pipeline("text2text-generation", model="dot-ammar/dotless_model-small")

2024-01-06 15:17:33.882450: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
2024-01-06 15:17:34.063028: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
All model checkpoin

In [18]:
#translator.save_pretrained('dotless-model-small')

In [113]:
def pipe_with_perams(input):
    return pipe(input, 
                num_return_sequences=10,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                max_new_tokens=40,
                num_beams=20,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
    

In [114]:
test = parse_text("قرأ أحمد القرآن")
input = test["dotless"]
target = test["clean"]
fullOutput = pipe_with_perams(input)
singleOutput = pipe(input)[0]['generated_text']

print("Input (dotless):", input)

print("Target (target):", target)

print("Single Output (dotted):", singleOutput)

print("Full Output (dotted):")
for i in range(len(fullOutput)):
    print(fullOutput[i]['generated_text'])

for i in range(len(fullOutput)):
    print(find_different_indices(target, fullOutput[i]['generated_text']))
    
print("Single Output Mistakes:", find_different_indices(target, singleOutput))



2024-01-06 16:20:52.173502: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 16:20:52.280260: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 16:20:52.541443: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 16:20:52.708862: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is und

Input (dotless): فرا احمد الفران
Target (target): قرأ أحمد القرآن
Single Output (dotted): قرأ أحمد القرآن
Full Output (dotted):
قرأ أحمد القرآن
قرأ احمد القرآن
قرأ أحمد القران
قرة أحمد القرآن
قرى أحمد القرآن
قري أحمد القرآن
قره أحمد القرآن
قرا أحمد القرآن
قرية أحمد القرآن
قرت أحمد القرآن
[]
[4]
[13]
[2]
[2]
[2]
[2]
[2]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[2]
Single Output Mistakes: []


2024-01-06 16:20:53.432464: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 16:20:53.490943: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 16:20:53.548096: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.


In [49]:
example1 = "فرا احمد الفران"
example2 = "لبسبفىد بكل حدىد"
example3 = "لدىنا عبر نرسل معلوماب الححر عبر البرىد الالكبرونى"


In [51]:
out_example1 = pipe(example1)
out_example2 = pipe(example2)
out_example3 = pipe(example3)

print(out_example1)
print(out_example2)
print(out_example3)

2024-01-06 15:25:28.235343: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/site-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2024-01-06 15:25:28.348933: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 15:25:28.508360: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out o

[{'generated_text': 'قرأ أحمد القرآن'}]
[{'generated_text': 'لتس اشت بكل جديد'}]
[{'generated_text': 'لدينا عبر نرسل معلومات الحجر عبر البريد الإلكتروني'}]


2024-01-06 15:25:29.700293: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
2024-01-06 15:25:29.757409: E ./tensorflow/core/kernels/cast_op_impl.h:64] IMPORTANT! The input tensor to Cast contains values out of range for the target type. This is undefined behavior and likely a bug in your model. A crash immediately after this under ubsan is expected.
